# Tutorial 05: Advanced Agent Techniques and Real-World Applications

In this tutorial, we'll explore advanced agent techniques in LangChain and apply them to create a sophisticated AI-powered research assistant.

In [3]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain.agents import Tool, AgentExecutor, LLMSingleActionAgent, AgentOutputParser
from langchain.prompts import StringPromptTemplate
from langchain import LLMChain
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from typing import List, Union
from langchain.schema import AgentAction, AgentFinish
import re
from langchain.agents import load_tools, initialize_agent, AgentType,create_react_agent, AgentExecutor


# Load environment variables
load_dotenv()
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')

# Initialize Groq LLM
llm = ChatGroq(model_name="qwen-2.5-32b")
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


## 1. Creating a Custom Agent with Specialized Capabilities

In [4]:
# Custom prompt template
template = """You are an AI research assistant designed to help with scientific literature analysis.
You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}"""

class CustomPromptTemplate(StringPromptTemplate):
    template: str
    tools: List[Tool]
    
    def format(self, **kwargs) -> str:
        intermediate_steps = kwargs.pop("intermediate_steps")
        thoughts = ""
        for action, observation in intermediate_steps:
            thoughts += action.log
            thoughts += f"\nObservation: {observation}\nThought: "
        kwargs["agent_scratchpad"] = thoughts
        kwargs["tools"] = "\n".join([f"{tool.name}: {tool.description}" for tool in self.tools])
        kwargs["tool_names"] = ", ".join([tool.name for tool in self.tools])
        return self.template.format(**kwargs)

class CustomOutputParser(AgentOutputParser):
    def parse(self, llm_output: str) -> Union[AgentAction, AgentFinish]:
        if "Final Answer:" in llm_output:
            return AgentFinish(
                return_values={"output": llm_output.split("Final Answer:")[-1].strip()},
                log=llm_output,
            )
        match = re.search(r"Action: (.*?)[\n]*Action Input: (.*)", llm_output, re.DOTALL)
        if not match:
            raise ValueError(f"Could not parse LLM output: `{llm_output}`")
        action = match.group(1).strip()
        action_input = match.group(2)
        return AgentAction(tool=action, tool_input=action_input.strip(" ").strip('"'), log=llm_output)


## 2. Implementing a """Multi-Agent""" System

In [5]:
# Load and process research papers
loader = DirectoryLoader('research_papers', glob="*.txt" ,loader_cls=TextLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)
vectorstore = FAISS.from_documents(texts, embedding)

# Define tools for the agents
search_tool = Tool(
    name="VectorStoreSearch",
    func=vectorstore.similarity_search,
    description="Searches the research papers for relevant information. Input should be a search query."
)

def summarize_tool_2(text):
    return llm.invoke(f"Summarize the following text in a concise manner: {text}")

summarize_tool = Tool(
    name="Summarize",
    func=summarize_tool_2,
    description="Summarizes a given text. Input should be the text to summarize."
)

def analize_tool_2(text):
    return llm.invoke(f"Analyze the following text and provide key insights: {text}") 

analyze_tool = Tool(
    name="Analyze",
    func=analize_tool_2,
    description="Analyzes a given text and provides key insights. Input should be the text to analyze."
)

tools = [search_tool, summarize_tool, analyze_tool]

prompt = CustomPromptTemplate(
    template=template,
    tools=tools,
    input_variables=["input", "intermediate_steps"]
)

output_parser = CustomOutputParser()

llm_chain = LLMChain(llm=llm, prompt=prompt)

research_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=[tool.name for tool in tools]
)
multi_tool_agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)


multi_tool_agent.invoke(
    "find me the most relevant information on the topic of Renewable Energy,summarize the information,analyze the information"
)

/tmp/ipykernel_4931/269614615.py:43: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(llm=llm, prompt=prompt)
/tmp/ipykernel_4931/269614615.py:45: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  research_agent = LLMSingleActionAgent(
/tmp/ipykernel_4931/2696



> Entering new AgentExecutor chain...
Thought: I know what to respond
Action:
```
{
  "action": "VectorStoreSearch",
  "action_input": "Renewable Energy"
}
```

Observation: [Document(id='e1222faf-cd5c-4f66-be80-a4c59921aacb', metadata={'source': 'research_papers/paper2.txt'}, page_content='# Advancements in Renewable Energy Technologies\n\n## Abstract\n\nThis paper examines recent advancements in renewable energy technologies, focusing on solar photovoltaics, wind energy, and energy storage systems. We analyze technological improvements, cost reductions, and integration challenges. Our findings suggest that renewable energy is increasingly competitive with fossil fuels and has the potential to significantly reduce global carbon emissions.\n\n## 1. Introduction\n\nThe transition to renewable energy sources is crucial for mitigating climate change and ensuring energy security. This paper reviews recent progress in key renewable technologies and discusses their potential for widespread

{'input': 'find me the most relevant information on the topic of Renewable Energy,summarize the information,analyze the information',
 'output': 'Recent advancements in renewable energy technologies, particularly in solar photovoltaics, wind energy, and energy storage, have enhanced performance and reduced costs, making renewables increasingly competitive with fossil fuels. These improvements have the potential to significantly reduce global carbon emissions. Key insights from the analysis include:\n\n1. **Technological Improvements**: Advancements in renewable energy technologies have improved their efficiency and reliability.\n2. **Cost Reduction**: Cost reductions have made renewable energy more affordable and competitive with fossil fuels.\n3. **Environmental Impact**: The shift towards renewable energy has the potential to significantly reduce global carbon emissions.\n4. **Ongoing Research and Development**: Continued research and development are essential to further enhance the 

## 3. Developing a Context-Aware Agent with Long-Term Memory

In [6]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

def converse_with_memory(input_text):
    response = conversation.predict(input=input_text)
    return response

memory_tool = Tool(
    name="ConversationMemory",
    func=converse_with_memory,
    description="Accesses the conversation history and provides context-aware responses. Input should be a question or statement."
)

tools.append(memory_tool)


context_aware_agent = LLMSingleActionAgent(
    llm_chain=llm_chain, 
    output_parser=output_parser,
    stop=["\nObservation:"], 
    allowed_tools=[tool.name for tool in tools]
)

context_aware_executor = AgentExecutor.from_agent_and_tools(agent=context_aware_agent, tools=tools, verbose=True)

/tmp/ipykernel_4931/1167170004.py:4: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_4931/1167170004.py:5: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  conversation = ConversationChain(


## 4. Building a Real-World Application: AI-Powered Research Assistant

In [7]:
def research_assistant(query):
    print(f"Research Query: {query}\n")
    response = context_aware_executor.invoke(query)
    print(f"Research Assistant's Response: {response}\n")
    return response

# Test the AI-powered research assistant
research_queries = [
    "What methods were used in the climate change study?",
    "What are the findings about coral reefs?",
    "What renewable energy technologies are discussed?",
    "What AI applications in medical imaging are mentioned?"
]

for query in research_queries:
    research_assistant(query)

Research Query: What methods were used in the climate change study?



> Entering new AgentExecutor chain...
I need to find information about a specific climate change study and the methods used. First, I should use VectorStoreSearch to find related research papers.
Action: VectorStoreSearch
Action Input: "climate change study methods"

Observation:[Document(id='7ee75b1c-b155-4a67-a572-e14e6f73e474', metadata={'source': 'research_papers/paper1.txt'}, page_content='# Climate Change Impacts on Global Ecosystems\n\n## Abstract\n\nThis paper reviews recent studies on the impacts of climate change on global ecosystems. We analyze data from various biomes, including tropical rainforests, coral reefs, and arctic tundra. Our findings indicate significant shifts in species distribution, phenology changes, and ecosystem degradation across multiple regions. We argue that urgent action is needed to mitigate these impacts and preserve biodiversity.\n\n## 1. Introduction\n\nClimate change, primarily

## Summary and Key Takeaways

In this tutorial, we've explored advanced agent techniques in LangChain and applied them to create a sophisticated AI-powered research assistant:

1. **Custom Agent Creation**: We developed a specialized agent for scientific literature analysis, demonstrating how to tailor agents for specific domains.

2. **Multi-Agent System**: By combining multiple tools (search, summarize, analyze), we created a versatile system capable of handling complex research tasks.

3. **Context-Aware Agent with Long-Term Memory**: We implemented a conversation memory, allowing the agent to maintain context across multiple interactions and provide more coherent and relevant responses over time.

4. **Real-World Application**: We built an AI-powered research assistant that can analyze scientific papers, extract key information, and provide insights on complex topics like climate change.

Key Takeaways:
- Advanced agents can be tailored for specific domains and tasks, greatly enhancing their effectiveness.
- Combining multiple tools and agent types allows for the creation of powerful, multi-functional AI systems.
- Implementing memory and context awareness significantly improves the quality and coherence of agent responses over time.
- Real-world applications of these techniques can lead to powerful AI assistants capable of handling complex, domain-specific tasks.

Next Steps:
- Experiment with different combinations of tools and agent types for your specific use cases.
- Explore ways to further enhance the agent's memory and context understanding capabilities.
- Consider integrating external APIs or databases to expand the agent's knowledge and capabilities.
